In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import os
import zipfile
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard  

# Configure parallelism parameters
tf.config.threading.set_intra_op_parallelism_threads(8) 

# Adjust based on your CPU
tf.config.threading.set_inter_op_parallelism_threads(8)  # Adjust based on your CPU

import zipfile

#Extract the zip files for training and testing data.

with zipfile.ZipFile('train.zip', 'r') as zip_ref:
    zip_ref.extractall('train_dataset')

with zipfile.ZipFile('test.zip', 'r') as zip_ref:
    zip_ref.extractall('test_dataset')   
    
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)   

# Create train generator
train_generator = train_data_gen.flow_from_directory(
    'train_dataset/train',  # Use the extracted directory
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)       

# Create test generator
test_generator = test_data_gen.flow_from_directory(
    'test_dataset/test',  # Use the extracted directory
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)                            

# Custom preprocessing function for sharpening using TensorFlow
def sharpen_image(image, label):
    # Apply Gaussian blur
    blurred = tf.image.adjust_contrast(image, contrast_factor=3)
    # Perform edge enhancement
    sharp_image = tf.clip_by_value(image * 2 - blurred, 0.0, 1.0)
    return sharp_image, label     

# Function to read and preprocess an image
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=1)  # Decode grayscale
    image = tf.image.resize(image, [48, 48])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    
    # Apply sharpening
    image, label = sharpen_image(image, label)
    
    return image, label     

 # Parameters
train_dir = "train_dataset/train"
validation_dir = "test_dataset/test"
batch_size = 32          



# Function to build dataset with one-hot encoding for labels
def build_dataset(directory, batch_size):
  # Create a dataset of file paths and labels
    file_paths = []
    labels = []
    label_map = {}
    for class_idx, class_name in enumerate(sorted(os.listdir(directory))):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            class_files = [os.path.join(class_dir, fname) for fname in os.listdir(class_dir)]
            file_paths.extend(class_files)
            labels.extend([class_idx] * len(class_files))
            label_map[class_idx] = class_name

    file_paths = tf.constant(file_paths)
    labels = tf.constant(labels)

    # One-hot encode the labels
    labels = tf.one_hot(labels, depth=5)  # Adjust 'depth' to match the number of classes
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset, label_map   

# Model Creation
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

optimizer = Adam(learning_rate=lr_schedule)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  
])      

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)      
# Create a TensorBoard callback for logging
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
tensorboard_callback = TensorBoard(log_dir=logdir)                                                                                                                                                              # Train the model
emotion_model_info = model.fit(
    train_generator,
    epochs=70,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[tensorboard_callback]
)

Found 23894 images belonging to 5 classes.
Found 5989 images belonging to 5 classes.
Epoch 1/70
374/374 [==============================] - 138s 364ms/step - loss: 1.5583 - accuracy: 0.3074 - val_loss: 1.4736 - val_accuracy: 0.3987
Epoch 2/70
374/374 [==============================] - 166s 445ms/step - loss: 1.3656 - accuracy: 0.4346 - val_loss: 1.2662 - val_accuracy: 0.4871
Epoch 3/70
374/374 [==============================] - 127s 341ms/step - loss: 1.2497 - accuracy: 0.4945 - val_loss: 1.1859 - val_accuracy: 0.5216
Epoch 4/70
374/374 [==============================] - 135s 361ms/step - loss: 1.1818 - accuracy: 0.5239 - val_loss: 1.1219 - val_accuracy: 0.5485
Epoch 5/70
374/374 [==============================] - 128s 343ms/step - loss: 1.1214 - accuracy: 0.5539 - val_loss: 1.0708 - val_accuracy: 0.5732
Epoch 6/70
374/374 [==============================] - 129s 344ms/step - loss: 1.0682 - accuracy: 0.5765 - val_loss: 1.0404 - val_accuracy: 0.5829
Epoch 7/70
374/374 [===================